# Lectura 51: Otras funciones

In [1]:
import polars as pl

df1 = pl.DataFrame(
    {
        'nombre': ['Jose'],
        'sexo': ['M']
    }
)

df2 = pl.DataFrame(
    {
        'nombre': ['Rosa', 'Teresa'],
        'sexo': ['F', 'F']
    }
)

# `concat`

Esta función nos permite combinar varios DataFrames, LazyFrames o Series en un solo objeto.

In [2]:
df = pl.concat([df1, df2])

print(df)

shape: (3, 2)
┌────────┬──────┐
│ nombre ┆ sexo │
│ ---    ┆ ---  │
│ str    ┆ str  │
╞════════╪══════╡
│ Jose   ┆ M    │
│ Rosa   ┆ F    │
│ Teresa ┆ F    │
└────────┴──────┘


Por defecto el concatenado se realiza de forma vertical pero con el parámetro `how` podemos especificar como deseamos que se realice la concatenación. Probemos concatenar los DataFrames anteriores horizontalmente.

In [3]:
df2 = df2.rename({'nombre': 'n', 'sexo': 's'})

df_horizontal = pl.concat([df1, df2], how='horizontal')

print(df_horizontal)

shape: (2, 4)
┌────────┬──────┬────────┬─────┐
│ nombre ┆ sexo ┆ n      ┆ s   │
│ ---    ┆ ---  ┆ ---    ┆ --- │
│ str    ┆ str  ┆ str    ┆ str │
╞════════╪══════╪════════╪═════╡
│ Jose   ┆ M    ┆ Rosa   ┆ F   │
│ null   ┆ null ┆ Teresa ┆ F   │
└────────┴──────┴────────┴─────┘


Existen otras estrategias de concatenación que se le pueden especificar al parámetro `how` las cuales la pueden consultar en la documentación.

[Consultar Documentación](https://docs.pola.rs/py-polars/html/reference/api/polars.concat.html#polars.concat)

# `collect_all`

Esta función nos permite colectar múltiples LazyFrames al mismo tiempo. Esto ejecutará todos los gráficos de cálculo en paralelo y nos devolverá los DataFrame en el mismo orden en que se le proporcionaron los LazyFrame.

In [4]:
from polars import col

query_1 = (
    pl.scan_parquet('./data/vuelos/vuelos.parquet')
    .bottom_k(20, by='DEPARTURE_TIME', descending=True, nulls_last=True)
)



query_2 = (
    pl.scan_parquet('./data/vuelos/vuelos.parquet')
    .filter((col('MONTH') > 6) & (col('DEPARTURE_DELAY') < 0))
    .select(
        col('MONTH'),
        col('DEPARTURE_DELAY')
    )
)

In [5]:
df1, df2 = pl.collect_all([query_1, query_2])

In [6]:
print(df1)

print(df2)

shape: (20, 31)
┌──────┬───────┬─────┬─────────────┬───┬──────────────┬──────────────┬──────────────┬──────────────┐
│ YEAR ┆ MONTH ┆ DAY ┆ DAY_OF_WEEK ┆ … ┆ SECURITY_DEL ┆ AIRLINE_DELA ┆ LATE_AIRCRAF ┆ WEATHER_DELA │
│ ---  ┆ ---   ┆ --- ┆ ---         ┆   ┆ AY           ┆ Y            ┆ T_DELAY      ┆ Y            │
│ i32  ┆ i32   ┆ i32 ┆ i32         ┆   ┆ ---          ┆ ---          ┆ ---          ┆ ---          │
│      ┆       ┆     ┆             ┆   ┆ i32          ┆ i32          ┆ i32          ┆ i32          │
╞══════╪═══════╪═════╪═════════════╪═══╪══════════════╪══════════════╪══════════════╪══════════════╡
│ 2015 ┆ 9     ┆ 8   ┆ 2           ┆ … ┆ 0            ┆ 0            ┆ 128          ┆ 0            │
│ 2015 ┆ 2     ┆ 17  ┆ 2           ┆ … ┆ 0            ┆ 3            ┆ 86           ┆ 0            │
│ 2015 ┆ 10    ┆ 21  ┆ 3           ┆ … ┆ null         ┆ null         ┆ null         ┆ null         │
│ 2015 ┆ 7     ┆ 8   ┆ 3           ┆ … ┆ 0            ┆ 0            ┆ 29  